In [197]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


def get_ret_year(mf_rets, st, ed):
    mf_rets_year = mf_rets[(mf_rets['date']<=ed) & (mf_rets['date']>=st)]
    date_ls = sorted(list(set(mf_rets_year['date'])))
    mf_rets_dicts = mf_rets_year.to_dict('records')
    rets_year_dict = {}
    for dict_i in mf_rets_dicts:
        crsp_fundno = dict_i['crsp_fundno']
        date = dict_i['date']
        ret = dict_i['mret']
        if crsp_fundno not in rets_year_dict:
            rets_year_dict[crsp_fundno] = {}
        rets_year_dict[crsp_fundno][date] = ret
    
    fundno_rets = {}
    NAN = float('nan')
    for fund_i in rets_year_dict:
        dict_i = rets_year_dict[fund_i]
        rest_ls = [NAN for i in range(len(date_ls))]
        for dete_i in dict_i:
            ind = date_ls.index(dete_i)
            rest_ls[ind] = dict_i[dete_i]
        fundno_rets[fund_i] = rest_ls
    fundno_rets_year1 = pd.DataFrame.from_dict(fundno_rets)
    return fundno_rets_year1


def get_qu_ls(quts, fund_retavg):
    ls = [[], [], [], [], [], [], [], [], [], []]
    for id_i in fund_retavg:
        avg = fund_retavg[id_i]
        for i in range(10):
            if (avg>=quts[i]) & (avg<quts[i+1]):
                ls[i].append(int(id_i))
    return ls
        
def get_group_ls(fundno_rets_year1):
    fundno_rets_year1 = fundno_rets_year1.dropna(axis=1, how='any')
    fund_retavg = fundno_rets_year1.mean().to_dict()
    ret_avgls = [fund_retavg[i] for i in fund_retavg]
    sum_len = len(fund_retavg_tp)
    # from low to high:
    quts = list(np.percentile(ret_avgls, [(i+1)*10 for i in range(9)]))
    quts.insert(0, -8888)
    quts.append(8888)
    portfolios_fundids = get_qu_ls(quts, fund_retavg)
    return portfolios_fundids


def get_pf_rets(pf_ids, fundno_rets_year2):
    for pf_i in pf_ids:
        ret_i = fundno_rets_year2[pf_i]
        

# 1 process data

In [88]:
mf_rets = pd.read_csv('./original_data/MFrets_1962to2012.csv')
mf_rets = mf_rets.dropna(axis=0, how='any')
years = list(mf_rets['year'])
months = list(mf_rets['month'].astype(str))
dates = []
for i in range(len(years)):
    date_i = str(years[i]) + str(months[i]).zfill(2)
    dates.append(date_i)
mf_rets['date'] = dates
mf_rets['mret'] = mf_rets['mret']*100
mf_rets = mf_rets.drop(['year', 'month'], axis=1)
mf_rets.to_csv('./MF_rets.csv')

In [61]:
mom = pd.read_csv('./original_data/Mom.csv')
df3 = pd.read_csv('./original_data/FF3.csv')
df3 = df3[df3['Unnamed: 0']>=192701]
df3 = df3.reset_index()
df3['Mom'] = mom['Mom']
dates1 = sorted(list(set(dates)))
df3['date'] = df3['Unnamed: 0']
df3 = df3.drop(['Unnamed: 0', 'index'], axis=1)
df3.to_csv('FF4.csv')

# 2 regression:

$$
\begin{gathered}
r_{i t}=\alpha_{i T}+\beta_{i T} \mathrm{VWRF}_{t}+e_{i t} \quad t=1,2, \cdots, T \\
r_{i t}=\alpha_{i T}+b_{i T} \mathrm{RMRF}_{t}+s_{i T} \mathrm{SMB}_{t}+h_{i T} \mathrm{HML}_{t}+e_{i t} \quad t=1,2, \cdots, T \\
r_{i t}=\alpha_{i T}+b_{i T} \mathrm{RMRF}_{t}+s_{i T} \mathrm{SMB}_{t}+h_{i T} \mathrm{HML}_{t}+p_{i T} \mathrm{PR} 1 \mathrm{YR}_{t}+e_{i t} \\
t=1,2, \cdots, T
\end{gathered}
$$

## 2.1 variable meaning:

- $r_{i t}$ is the return on a portfolio in excess of the one-month T-bill return;
- VWRF is the excess return on the CRSP value-weighted portfolio of all NYSE;
- RMRF is the excess return on a value-weighted aggregate market proxy;
- SMB, HML, PR1YR are returns on valueweighted, zero-investment, factor-mimicking portfolios for size, book-to-market equity, and one-year momentum in stock returns.


## 2.2 portfolio  construction:

On January 1 of each year, I form ten equal-weighted portfolios of mutual funds, using reported returns.  I hold the portfolios for one year, then reform them. This yields a time series of monthly returns on each decile portfolio from 1963 to 1993. Funds that disappear during the course of the year are included in the equal-weighted average until they disappear, then the portfolio weights are readjusted appropriately.

Mutual funds are sorted on January 1 each year from 1963 to 1993 into decile portfolios based on their previous calendar year's return. The portfolios are equally weighted monthly so the weights are readjusted whenever a fund disappears. Funds with the highest past one-year return comprise decile 1 and funds with the lowest comprise decile 10. Deciles 1 and 10 are further subdivided into thirds on the same measure. VWRF is the excess return on the CRSP value-weight market proxy.

RMRF, SMB, and HML are Fama and French's (1993) market proxy and factor-mimicking portfolios for size and book-to-market equity. PR1YR is a factor-mimicking portfolio for one-year return momentum. Alpha is the intercept of the Model. The t-statistics are in parentheses.

In [89]:
ff4_df = pd.read_csv('./FF4.csv')
ff4_df = ff4_df.drop('Unnamed: 0', axis=1)
mf_rets = pd.read_csv('./MF_rets.csv')
mf_rets = mf_rets.drop('Unnamed: 0', axis=1)

In [200]:
st = 196401
ed = 196412
fundno_rets_year1 = get_ret_year(mf_rets, st, ed)
pf_group_ls = get_group_ls(fundno_rets_year1)

In [201]:
st = 196501
ed = 196512
fundno_rets_year2 = get_ret_year(mf_rets, st, ed)

In [203]:
fundno_rets_year2[pf_group_ls[0]]

,327,7235,26998,32675,33392,34130,35061
0,1.9967,5.0489,6.7485,4.7000,5.6000,9.4000,6.6000
1,-1.7651,-2.1823,6.0345,-3.3429,4.9242,2.9250,4.9719
2,-1.4048,-5.0568,-1.6260,-3.5573,-1.0830,-0.3552,-0.9830
3,-0.7621,2.7720,2.7548,2.4590,3.9234,6.7736,5.6859
4,0.4674,-7.4848,-0.2681,-0.6000,-1.4925,-2.5042,-0.8540
5,-3.6225,-3.8639,-9.9462,-9.7586,-10.2496,-11.3870,-10.3359
6,1.0000,-1.1407,2.9851,-2.2297,2.2840,3.7681,3.3622
7,1.0584,-2.0769,6.3768,-2.5086,6.8932,8.3799,6.2268
8,0.5405,5.8916,7.3569,4.6784,1.0899,4.2955,4.3745
9,-0.2352,-3.7834,4.8223,-1.3408,3.8634,6.1779,9.0528
